In [1]:
#mouse 测试版
import serial
import pyautogui
import time

# 初始化串口连接
arduino = serial.Serial(port='COM3', baudrate=115200, timeout=0.1)  # 修改为你的串口号

# 鼠标移动灵敏度
sensitivity = 50

# # 等待串口稳定
# time.sleep(2)

print("开始读取 Arduino 数据并控制鼠标...")

try:
    while True:
        # 从串口读取数据
        data = arduino.readline().decode().strip()
        if not data:
            continue  # 跳过空行

        # 打印接收到的数据（用于调试）
        print(f"收到的数据: '{data}'")

        try:
            # 确保数据格式为 "X轴,Y轴,按键"
            if ',' in data and len(data.split(",")) == 3:
                x_val, y_val, button = map(int, data.split(","))

                # 映射摇杆值到鼠标移动
                x_move = (x_val - 512) / sensitivity
                y_move = -(y_val - 512) / sensitivity

                # 移动鼠标
                pyautogui.moveRel(x_move, -y_move)  # Y轴方向反转

                # 按键模拟（左键点击）
                if button == 0:  # 按键按下
                    pyautogui.click()

        except ValueError:
            print(f"数据解析错误: {data}")  # 打印错误数据以调试

except KeyboardInterrupt:
    print("程序已终止")

finally:
    arduino.close()


开始读取 Arduino 数据并控制鼠标...
收到的数据: '609,501,1'
收到的数据: '1018,501,1'
收到的数据: '1020,501,1'
收到的数据: '1020,501,1'
收到的数据: '1020,501,1'
收到的数据: '1018,0,1'
收到的数据: '1018,123,1'
收到的数据: '909,431,1'
收到的数据: '913,431,1'
收到的数据: '1019,328,1'
收到的数据: '1018,0,1'
收到的数据: '1018,0,1'
收到的数据: '1018,0,1'
收到的数据: '1019,0,1'
收到的数据: '1016,1,1'
收到的数据: '901,501,1'
收到的数据: '1018,501,1'
收到的数据: '1018,497,1'
收到的数据: '1019,327,1'
收到的数据: '1018,191,1'
收到的数据: '1018,189,1'
收到的数据: '1020,190,1'
收到的数据: '1020,210,1'
收到的数据: '512,395,1'
收到的数据: '1018,502,1'
收到的数据: '1018,467,1'
收到的数据: '1017,0,1'
收到的数据: '1015,0,1'
收到的数据: '512,0,1'
收到的数据: '189,0,1'
收到的数据: '138,501,1'
收到的数据: '512,674,1'
收到的数据: '512,1017,1'
收到的数据: '512,1018,1'
收到的数据: '938,1018,1'
收到的数据: '1017,1016,1'
收到的数据: '1017,501,1'
收到的数据: '1016,0,1'
收到的数据: '1016,0,1'
收到的数据: '530,487,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,405,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '511,0,1'
收到的数据: '1016,0,1'
收到

In [7]:
import pygame
import sys
import random
import serial
import time
import subprocess

# 初始化串口
try:
    arduino = serial.Serial(port='COM3', baudrate=115200, timeout=0.1)  # 修改为你的串口号
    time.sleep(2)
except serial.SerialException as e:
    print(f"无法连接到 Arduino: {e}")
    arduino = None

# 初始化 Pygame
pygame.init()

# 设置屏幕尺寸和标题
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("打砖块游戏")

# 定义颜色
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)  # 分裂道具颜色

# 定义玩家板子参数
paddle_width = 100
paddle_height = 10
paddle_x = screen_width // 2 - paddle_width // 2
paddle_y = screen_height - 20
paddle_speed = 14

# 定义球参数
ball_size = 10
balls = [{'x': screen_width // 2, 'y': screen_height // 2, 'speed_x': 8, 'speed_y': -8}]

# 定义砖块参数
brick_rows = 5
brick_columns = 10
brick_width = 70
brick_height = 20
brick_padding = 10
brick_offset_top = 50
brick_offset_left = 35

# 定义分裂道具参数
powerups = []  # 存储当前掉落的分裂道具
powerup_size = 20
powerup_speed = 5
powerup_chance = 0.3  # 分裂道具掉落的概率（30%）

# 创建砖块
bricks = []
for row in range(brick_rows):
    for col in range(brick_columns):
        brick_x = brick_offset_left + col * (brick_width + brick_padding)
        brick_y = brick_offset_top + row * (brick_height + brick_padding)
        bricks.append(pygame.Rect(brick_x, brick_y, brick_width, brick_height))

# 初始化分数
score = 0
font = pygame.font.SysFont(None, 36)

# 游戏主循环
clock = pygame.time.Clock()
running = True

def end_game():
    """结束游戏并跳转到游戏结束窗口"""
    if arduino and arduino.is_open:
            arduino.close()
            print("串口已关闭")
            print("返回主菜单")
    pygame.quit()
    subprocess.Popen(['python', 'game_over_ui.py', '打砖块游戏', str(score)])
    sys.exit()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 从 Arduino 接收数据
    try:
        data = arduino.readline().decode().strip()
        if data and ',' in data:
            parts = data.split(",")
            if len(parts) == 3:  # 确保数据格式正确
                x_val, y_val, button = map(int, parts)

                # 根据 X 值控制板子移动
                if x_val < 400:  # 偏左
                    paddle_x -= paddle_speed
                elif x_val > 600:  # 偏右
                    paddle_x += paddle_speed

    except Exception as e:
        print(f"读取数据错误: {e}")
        pass

    # 限制板子移动范围
    paddle_x = max(0, min(screen_width - paddle_width, paddle_x))

    # 更新所有小球的位置
    for ball in balls[:]:  # 遍历当前所有小球
        ball['x'] += ball['speed_x']
        ball['y'] += ball['speed_y']

        # 球碰到墙壁反弹
        if ball['x'] <= 0 or ball['x'] >= screen_width - ball_size:
            ball['speed_x'] = -ball['speed_x']
        if ball['y'] <= 0:
            ball['speed_y'] = -ball['speed_y']

        # 球碰到板子反弹
        paddle_rect = pygame.Rect(paddle_x, paddle_y, paddle_width, paddle_height)
        if paddle_rect.colliderect(pygame.Rect(ball['x'], ball['y'], ball_size, ball_size)):
            ball['speed_y'] = -ball['speed_y']

        # 球碰到砖块
        for brick in bricks[:]:
            if brick.colliderect(pygame.Rect(ball['x'], ball['y'], ball_size, ball_size)):
                bricks.remove(brick)
                ball['speed_y'] = -ball['speed_y']
                score += 1
                # 掉落分裂道具的概率
                if random.random() < powerup_chance:
                    powerups.append({'x': brick.x + brick_width // 2 - powerup_size // 2,
                                     'y': brick.y + brick_height // 2,
                                     'speed_y': powerup_speed})

        # 检查球是否掉出屏幕
        if ball['y'] > screen_height:
            balls.remove(ball)
            if len(balls) == 0:  # 如果所有球都掉出屏幕
                print("游戏结束！")
                running = False

    # 更新分裂道具的位置
    for powerup in powerups[:]:
        powerup['y'] += powerup['speed_y']

        # 检查是否被玩家接住
        if paddle_rect.colliderect(pygame.Rect(powerup['x'], powerup['y'], powerup_size, powerup_size)):
            powerups.remove(powerup)
            # 分裂出两个新球
            for ball in balls[:]:
                balls.append({'x': ball['x'], 'y': ball['y'], 'speed_x': -ball['speed_x'], 'speed_y': ball['speed_y']})
                balls.append({'x': ball['x'], 'y': ball['y'], 'speed_x': ball['speed_x'], 'speed_y': -ball['speed_y']})

        # 如果道具掉到屏幕下方，移除它
        elif powerup['y'] > screen_height:
            powerups.remove(powerup)

    # 如果所有砖块被清除，结束游戏
    if not bricks:
        print("游戏胜利！")
        running = False

    # 绘制屏幕内容
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLUE, paddle_rect)  # 绘制板子
    for ball in balls:
        pygame.draw.ellipse(screen, RED, (ball['x'], ball['y'], ball_size, ball_size))  # 绘制所有小球

    # 绘制砖块
    for brick in bricks:
        pygame.draw.rect(screen, BLACK, brick)

    # 绘制分裂道具
    for powerup in powerups:
        pygame.draw.rect(screen, YELLOW, (powerup['x'], powerup['y'], powerup_size, powerup_size))

    # 显示分数
    score_text = font.render(f"分数: {score}", True, BLACK)
    screen.blit(score_text, (10, 10))

    # 更新屏幕
    pygame.display.flip()
    clock.tick(60)

# 游戏结束，跳转到游戏结束窗口
end_game()


游戏结束！
串口已关闭
返回主菜单


SystemExit: 

In [1]:
import pygame
import sys
import random
import serial
import time

# 初始化串口
arduino = serial.Serial(port='COM3', baudrate=115200, timeout=0.1)  # 修改為你的串口號
time.sleep(2)

# 初始化 Pygame
pygame.init()

# 設置屏幕尺寸和標題
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("接物遊戲")

# 定義顏色
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)

# 定義玩家參數
player_size = 100
player_x = screen_width // 2 - player_size // 2
player_y = screen_height - 20
player_speed = 12  # 調整靈敏度

# 定義物品參數
item_size = 50
item_x = random.randint(0, screen_width - item_size)
item_y = 0 - item_size
item_speed = 20
item_color = random.choice([GREEN, RED])

# 初始化分數
score = 0
font = pygame.font.SysFont(None, 36)

# 遊戲主循環
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 從 Arduino 讀取數據
    try:
        data = arduino.readline().decode().strip()
        if data and ',' in data:
            parts = data.split(",")
            if len(parts) == 3:  # 確保數據格式正確
                x_val, y_val, button = map(int, parts)

                # 將 X 軸數據轉換為板子移動
                if x_val < 400:  # 偏左
                    player_x -= player_speed
                elif x_val > 600:  # 偏右
                    player_x += player_speed
    except Exception as e:
        print(f"讀取數據錯誤: {e}")
        pass

    # 限制板子位置不超出邊界
    if player_x < 0:
        player_x = 0
    if player_x > screen_width - player_size:
        player_x = screen_width - player_size

    # 更新物品位置
    item_y += item_speed
    if item_y > screen_height:
        item_y = 0 - item_size
        item_x = random.randint(0, screen_width - item_size)
        item_color = random.choice([GREEN, RED])

    # 檢測接住
    if (player_x < item_x + item_size and
        player_x + player_size > item_x and
        player_y < item_y + item_size and
        player_y + 10 > item_y):
        if item_color == GREEN:
            score += 1
        elif item_color == RED:
            print("遊戲結束！")
            running = False
        item_y = 0 - item_size
        item_x = random.randint(0, screen_width - item_size)
        item_color = random.choice([GREEN, RED])

    # 繪製屏幕內容
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLUE, (player_x, player_y, player_size, 10))  # 玩家為長條
    pygame.draw.rect(screen, item_color, (item_x, item_y, item_size, item_size))

    # 顯示分數
    score_text = font.render(f"分數: {score}", True, BLACK)
    screen.blit(score_text, (10, 10))

    # 更新屏幕
    pygame.display.flip()
    clock.tick(30)

pygame.quit()
sys.exit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

c:\Users\Owner\anaconda3\envs\hw\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import pygame
import sys
import random  # 修正：导入 random 模块
import serial
import time

# 初始化串口
arduino = serial.Serial(port='COM3', baudrate=115200, timeout=0.1) # 修改为你的串口号
time.sleep(2)
# 初始化 Pygame
pygame.init()

# 设置屏幕尺寸和标题
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("接物小游戏")

# 定义颜色
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)

# 定义玩家参数
player_size = 100
player_x = screen_width // 2 - player_size // 2
player_y = screen_height - 20
player_speed = 24

# 定义物品参数
item_size = 50
item_x = random.randint(0, screen_width - item_size)
item_y = 0 - item_size
item_speed = 20
item_color = random.choice([GREEN, RED])  # 随机选择物品颜色

# 初始化分数
score = 0
font = pygame.font.SysFont(None, 36)

# 游戏主循环
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    try:
            data = arduino.readline().decode().strip()
            if data and ',' in data:
                parts = data.split(",")
                if len(parts) == 3:  # 确保数据格式正确
                    x_val, y_val, button = map(int, parts)

                    # 根据 X 值控制板子移动
                    if x_val < 400:  # 偏左
                        player_x -= player_speed
                    elif x_val > 600:  # 偏右
                        player_x += player_speed
    except Exception as e:
        print(f"读取数据错误: {e}")
        pass

    # 获取按键输入
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and player_x > 0:
        player_x -= player_speed
    if keys[pygame.K_RIGHT] and player_x < screen_width - player_size:
        player_x += player_speed

    # 更新物品位置
    item_y += item_speed
    if item_y > screen_height:
        item_y = 0 - item_size
        item_x = random.randint(0, screen_width - item_size)
        item_color = random.choice([GREEN, RED])  # 重新随机选择颜色

    # 检测接住
    if (player_x < item_x + item_size and
        player_x + player_size > item_x and
        player_y < item_y + item_size and
        player_y + 10 > item_y):
        if item_color == GREEN:
            score += 1
        elif item_color == RED:
            print("游戏结束！")
            running = False
        item_y = 0 - item_size
        item_x = random.randint(0, screen_width - item_size)
        item_color = random.choice([GREEN, RED])

    # 绘制屏幕内容
    screen.fill(WHITE)
    pygame.draw.rect(screen, BLUE, (player_x, player_y, player_size, 10))  # 玩家为长条
    pygame.draw.rect(screen, item_color, (item_x, item_y, item_size, item_size))

    # 显示分数
    score_text = font.render(f"分数: {score}", True, BLACK)
    screen.blit(score_text, (10, 10))

    # 更新屏幕
    pygame.display.flip()
    clock.tick(30)

pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel
from PyQt5.QtCore import Qt

class MainMenu(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        # 设置窗口标题和大小
        self.setWindowTitle("主菜单")
        self.setGeometry(300, 300, 400, 300)

        # 设置背景图片
        self.setStyleSheet("background-image: url('bcaaag.jpg'); background-repeat: no-repeat; background-position: center;")

        # 垂直布局
        layout = QVBoxLayout()

        # 标题
        title = QLabel("请选择一个程序")
        title.setStyleSheet("font-size: 20px; font-weight: bold; color: white; background-color: rgba(0, 0, 0, 0.5);")  # 半透明背景
        title.setAlignment(Qt.AlignCenter)
        layout.addWidget(title)

        # 按钮
        btn_style = """
            QPushButton {
                font-size: 16px;
                color: black;
                background-color: white;
                border: 1px solid black;
                border-radius: 5px;
                padding: 5px;
            }
            QPushButton:hover {
                background-color: lightgray;
            }
        """
        btn1 = QPushButton("Arduino 鼠标控制")
        btn1.setStyleSheet(btn_style)
        btn1.clicked.connect(lambda: self.run_program('arduino_mouse_control.py'))
        layout.addWidget(btn1)

        btn2 = QPushButton("打砖块游戏")
        btn2.setStyleSheet(btn_style)
        btn2.clicked.connect(lambda: self.run_program('brick_breaker.py'))
        layout.addWidget(btn2)

        btn3 = QPushButton("接物小游戏")
        btn3.setStyleSheet(btn_style)
        btn3.clicked.connect(lambda: self.run_program('catch_game.py'))
        layout.addWidget(btn3)

        btn_exit = QPushButton("退出")
        btn_exit.setStyleSheet(btn_style)
        btn_exit.clicked.connect(self.close)
        layout.addWidget(btn_exit)

        # 设置布局
        self.setLayout(layout)

    def run_program(self, file_name):
        import subprocess
        try:
            # 使用 subprocess 启动子程序
            subprocess.Popen(['python', file_name])
        except Exception as e:
            print(f"运行 {file_name} 时出错：{e}")

if __name__ == '__main__':
    app = QApplication([])
    menu = MainMenu()
    menu.show()
    app.exec_()



In [1]:
import serial
import pyautogui
import time
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout
from threading import Thread

# 初始化串口连接
arduino = serial.Serial(port='COM3', baudrate=115200, timeout=0.1)  # 修改为你的串口号

# 鼠标移动灵敏度
sensitivity = 50

class MouseControlApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.running = True

    def initUI(self):
        # 设置窗口标题和大小
        self.setWindowTitle("鼠标控制")
        self.setGeometry(100, 100, 200, 100)

        # 创建返回按钮
        layout = QVBoxLayout()
        self.return_button = QPushButton("返回主菜单")
        self.return_button.clicked.connect(self.return_to_menu)
        layout.addWidget(self.return_button)

        self.setLayout(layout)

    def return_to_menu(self):
        # 停止鼠标控制线程
        self.running = False
        arduino.close()
        print("返回主菜单")
        self.close()

def mouse_control(app):
    print("开始读取 Arduino 数据并控制鼠标...")
    try:
        while app.running:
            # 从串口读取数据
            data = arduino.readline().decode().strip()
            if not data:
                continue  # 跳过空行

            # 打印接收到的数据（用于调试）
            print(f"收到的数据: '{data}'")

            try:
                # 确保数据格式为 "X轴,Y轴,按键"
                if ',' in data and len(data.split(",")) == 3:
                    x_val, y_val, button = map(int, data.split(","))

                    # 映射摇杆值到鼠标移动
                    x_move = (x_val - 512) / sensitivity
                    y_move = -(y_val - 512) / sensitivity

                    # 移动鼠标
                    pyautogui.moveRel(x_move, -y_move)  # Y轴方向反转

                    # 按键模拟（左键点击）
                    if button == 0:  # 按键按下
                        pyautogui.click()

            except ValueError:
                print(f"数据解析错误: {data}")  # 打印错误数据以调试

    except KeyboardInterrupt:
        print("程序已终止")
    finally:
        arduino.close()

if __name__ == "__main__":
    # 创建 PyQt 应用
    app = QApplication(sys.argv)
    main_window = MouseControlApp()

    # 创建并启动鼠标控制线程
    thread = Thread(target=mouse_control, args=(main_window,))
    thread.start()

    # 显示主窗口
    main_window.show()
    sys.exit(app.exec_())


开始读取 Arduino 数据并控制鼠标...
收到的数据: '491,480,1'
收到的数据: '1015,1007,1'
收到的数据: '1014,1015,1'
收到的数据: '1008,1009,1'
收到的数据: '986,986,1'
收到的数据: '1017,1018,1'
收到的数据: '1015,1015,1'
收到的数据: '1015,1015,1'
收到的数据: '1014,1015,1'
收到的数据: '1014,1013,1'
收到的数据: '1013,1013,1'
收到的数据: '1010,1012,1'
收到的数据: '1012,1013,1'
收到的数据: '1013,1012,1'
收到的数据: '1013,1013,1'
收到的数据: '1012,1014,1'
收到的数据: '1015,1015,1'
收到的数据: '1015,1015,1'
收到的数据: '1015,1015,1'
收到的数据: '1014,1015,1'
收到的数据: '1014,1015,1'
收到的数据: '1013,1016,1'
收到的数据: '1010,1011,1'
收到的数据: '1009,1012,1'
收到的数据: '500,490,1'
收到的数据: '511,688,1'
收到的数据: '1019,1018,1'
收到的数据: '1020,1018,1'
收到的数据: '1019,1021,1'
收到的数据: '844,1020,1'
收到的数据: '507,1008,1'
收到的数据: '510,1013,1'
收到的数据: '509,1014,1'
收到的数据: '509,1013,1'
收到的数据: '509,1015,1'
收到的数据: '711,1015,1'
收到的数据: '1017,1016,1'
收到的数据: '1017,1018,1'
收到的数据: '1018,1017,1'
收到的数据: '1016,1015,1'
收到的数据: '1016,1014,1'
收到的数据: '1013,1013,1'
收到的数据: '1010,1009,1'
收到的数据: '1011,1012,1'
收到的数据: '721,1009,1'
收到的数据: '508,1010,1'
收到的数据: '507,1009,1'
收到的数据: 

SystemExit: 0

c:\Users\Owner\anaconda3\envs\hw\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import subprocess
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel
from PyQt5.QtCore import Qt

class MainMenu(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.running_process = None  # 当前运行的子程序

    def initUI(self):
        self.setWindowTitle("功能选择菜单")
        self.setGeometry(300, 300, 400, 300)

        layout = QVBoxLayout()

        # 添加按钮
        btn_mouse = QPushButton("启动鼠标控制")
        btn_mouse.clicked.connect(lambda: self.start_function("mouse.py"))
        layout.addWidget(btn_mouse)

        btn_menu = QPushButton("启动菜单功能")
        btn_menu.clicked.connect(lambda: self.start_function("menu.py"))
        layout.addWidget(btn_menu)

        btn_exit = QPushButton("退出")
        btn_exit.clicked.connect(self.close_application)
        layout.addWidget(btn_exit)

        self.setLayout(layout)

    def start_function(self, script_name):
        # 停止当前运行的子程序
        if self.running_process:
            self.running_process.terminate()

        # 启动新的子程序
        try:
            self.running_process = subprocess.Popen(['python', script_name])
        except Exception as e:
            print(f"启动 {script_name} 出错：{e}")

    def close_application(self):
        # 退出前关闭子程序
        if self.running_process:
            self.running_process.terminate()
        self.close()

if __name__ == "__main__":
    app = QApplication([])
    menu = MainMenu()
    menu.show()
    app.exec_()
